In [ ]:
import pandas as pd
import seaborn as sns
import datetime
import matplotlib.pyplot as plt
import numpy as np
import zipfile
import plotly.graph_objects as go
import datetime
from sklearn.cluster import KMeans

# Introducao
## Bases Utilizadas

- [Base BNDES (Desembolsos)](https://dadosabertos.bndes.gov.br/dataset/desembolsos-mensais/resource/aac5c7bd-5d18-41c3-9720-52d20470427e) (Carregada por meio da API do portal de dados abertos do BNDES)

- [Base WDI (Países)](https://datatopics.worldbank.org/world-development-indicators/) (Carregada localmente)


# Base Desembolsos Mensais

In [ ]:
url_desembolsos = "https://dadosabertos.bndes.gov.br/dataset/c529f6a7-5ce4-4378-a585-92cbf53af710/resource/aac5c7bd-5d18-41c3-9720-52d20470427e/download/desembolsos-mensais.csv"

df_desembolsos = pd.read_csv(url_desembolsos, sep=';',decimal=",")


# Tratandos os dados dos desembolsos

In [ ]:
df_desembolsos['date'] = pd.to_datetime(df_desembolsos['ano'].astype(str) + '-' + df_desembolsos['mes'].astype(str) + '-' + "01")
df_desembolsos.loc[:,['regiao', 'uf', 'municipio']] = df_desembolsos.loc[:,['regiao', 'uf', 'municipio']].replace('\s*$', '',regex=True)
df_desembolsos.sort_values(by=['date'], inplace=True)
df_desembolsos.head()

# Agrupando os dados do BNDES por subsetor e plotando

In [ ]:
desembolsosBySubsetor = pd.DataFrame(df_desembolsos.groupby(by=["ano", "subsetor_bndes"]).sum().desembolsos_reais).unstack()
desembolsosBySubsetor = desembolsosBySubsetor.droplevel(0, axis=1)


fig = go.Figure([go.Scatter(x=desembolsosBySubsetor.index, y=desembolsosBySubsetor[i].values, name=i) for i in desembolsosBySubsetor.columns])
fig.show()

# Calculando a porcentagem de investimento em cada área e plotando

In [ ]:
subsetores = ['AGROPECUÁRIA', 'ALIMENTO E BEBIDA']

porcentoSubsetor = desembolsosBySubsetor.div(desembolsosBySubsetor.sum(axis=1), axis=0)
porcentoSubsetor = porcentoSubsetor.apply(lambda x: x*100)

fig = go.Figure([go.Scatter(x=porcentoSubsetor.index, y=porcentoSubsetor[i].values, name=i) for i in subsetores])
fig.show()

# Base WDI

In [ ]:
wdi_zip = zipfile.ZipFile('../data/WDI_csv.zip')
df_wdi = pd.read_csv(wdi_zip.open('WDIData.csv'))

In [ ]:
df_wdi.head(3)

Vamos checar agora quais anos sao cobertos pelos dados do BNDES para filtrarmos a base WDI com dados de nosso interesse.

In [ ]:
lista_anos_interesse = df_desembolsos['ano'].sort_values().unique().astype(str)

print(lista_anos_interesse)

Vemos que sao dados a partir de 1995 com termino em 2020. Com essas informacoes, vamos filtrar as colunas da base wdi. Vamos manter as colunas contendo o nome do pais, codigo do pais e nome do indicador (Country Name, Country Code, Indicator Name)

In [ ]:
df_wdi[lista_anos_interesse]

In [ ]:
cols_wdi_indicadores =["Chemicals (% of value added in manufacturing)",
                        "Agriculture, forestry, and fishing, value added (% of GDP)",                        
                        "Industry (including construction), value added (% of GDP)",
                        "Services, value added (% of GDP)",
                        "Manufacturing, value added (% of GDP)",
                        "Food, beverages and tobacco (% of value added in manufacturing)",
                        "Textiles and clothing (% of value added in manufacturing)", 
                        "Machinery and transport equipment (% of value added in manufacturing)",
                        "Other manufacturing (% of value added in manufacturing)",
                        "GDP per capita growth (annual %)"]

In [ ]:
for i in cols_wdi_indicadores:
    print(i)

In [ ]:
for i in df_desembolsos['subsetor_bndes'].unique():
    print(i)

In [ ]:
cols_desembolsos_indicadores = ['QUÍMICA E PETROQUÍMICA',
                                'AGROPECUÁRIA',
                                'CONSTRUÇÃO',
                                'COMÉRCIO E SERVIÇOS',
                                'OUTROS TRANSPORTES',
                                'MATERIAL DE TRANSPORTE',
                                'ALIMENTO E BEBIDA',
                                'ENERGIA ELÉTRICA',
                                'CELULOSE E PAPEL',
                                'TELECOMUNICAÇÕES',
                                'METALURGIA E PRODUTOS',
                                'EXTRATIVA',
                                'TRANSPORTE FERROVIÁRIO',
                                'ATV. AUX. TRANSPORTES',
                                'MECÂNICA',                                
                                'SERV. UTILIDADE PÚBLICA',
                                'TÊXTIL E VESTUÁRIO',
                                'OUTRAS',
                                'TRANSPORTE RODOVIÁRIO',
                                'OUTROS']

In [ ]:
df_desembolsos['subsetor_cnae_agrupado'].sort_values().unique()

Selecionar melhor estrutura para o dicionario

In [ ]:
relacao_01 = {"quimica" : ["QUÍMICA E PETROQUÍMICA", "Chemicals (% of value added in manufacturing)"], 
              "agropecuaria": ["AGROPECUÁRIA", "Agriculture, forestry, and fishing, value added (% of GDP)"], 
              "industria": [["CELULOSE E PAPEL","METALURGIA E PRODUTOS","EXTRATIVA", "CONSTRUÇÃO"], "Industry (including construction), value added (% of GDP)"],
              "servicos":[["COMÉRCIO E SERVIÇOS", "ENERGIA ELÉTRICA", "TELECOMUNICAÇÕES", "TRANSPORTE FERROVIÁRIO", "ATV. AUX. TRANSPORTES", "SERV. UTILIDADE PÚBLICA", "TRANSPORTE RODOVIÁRIO"], "Services, value added (% of GDP)"],
              "maquinario":[["MATERIAL DE TRANSPORTE", "MECÂNICA"], "Machinery and transport equipment (% of value added in manufacturing)"],
              "alimentacao_tabaco": ["ALIMENTO E BEBIDA", "Food, beverages and tobacco (% of value added in manufacturing)"],
              "textil": ["TÊXTIL E VESTUÁRIO", "Textiles and clothing (% of value added in manufacturing)"] }

relacao_02 = {"Chemicals (% of value added in manufacturing)" : ["QUÍMICA E PETROQUÍMICA"], 
              "Agriculture, forestry, and fishing, value added (% of GDP)": ["AGROPECUÁRIA"], 
              "Industry (including construction), value added (% of GDP)": ["CELULOSE E PAPEL","METALURGIA E PRODUTOS","EXTRATIVA", "CONSTRUÇÃO"],
              "Services, value added (% of GDP)":[["COMÉRCIO E SERVIÇOS", "ENERGIA ELÉTRICA", "TELECOMUNICAÇÕES", "TRANSPORTE FERROVIÁRIO", "ATV. AUX. TRANSPORTES", "SERV. UTILIDADE PÚBLICA", "TRANSPORTE RODOVIÁRIO"]],
              "Machinery and transport equipment (% of value added in manufacturing)":["MATERIAL DE TRANSPORTE", "MECÂNICA"],
              "Food, beverages and tobacco (% of value added in manufacturing)": ["ALIMENTO E BEBIDA"],
              "Textiles and clothing (% of value added in manufacturing)": ["TÊXTIL E VESTUÁRIO"]}

# Agrupando os dados de desembolso de acordo com os setores definidos no dicionário relacao_01

In [ ]:
percentageSeries = [porcentoSubsetor[relacao_01[i][0]].T.sum() if isinstance(relacao_01[i][0], list) else porcentoSubsetor[relacao_01[i][0]] for i in relacao_01.keys()]

df_desembolsos_organizado = pd.DataFrame(percentageSeries, index=relacao_01.keys()).T
df_desembolsos_organizado.head()

# Tratando os dados da base WDI para usar-los na clusterização

In [ ]:
wdi_t = df_wdi[df_wdi['Indicator Name'].str.match(r'^.*(% of GDP).*$')].T
idx = pd.IndexSlice

wdi_t.columns = pd.MultiIndex.from_product([wdi_t.loc['Country Name'].unique(), wdi_t.loc['Indicator Name'].unique()])
wdi_t.drop(labels=['Country Name','Indicator Name', 'Country Code'], axis=0, inplace=True)
wdi_t.drop(index=['Indicator Code','Unnamed: 65'], inplace=True)
wdi_t.index = list(wdi_t.index.str.extract(r'([0-9]{4})').values.astype(int).reshape((1,-1)))
wdi_t.replace(regex=r'^.*$', value=np.nan, inplace=True)
wdi_t = wdi_t.apply(pd.to_numeric)
wdi_t.head()

In [ ]:
countries = ['Brazil', 'Spain', 'Mexico', 'Turkey', 'Germany','World']

fig = go.Figure([go.Scatter(
    x=wdi_t.loc[:,idx[:,'Agriculture, forestry, and fishing, value added (% of GDP)']].index.levels[0], 
    y=wdi_t.loc[:,idx[:,'Agriculture, forestry, and fishing, value added (% of GDP)']][i].values.flatten(), name=i) 
    for i in countries])
fig.show()

# Construindo o dataframe concatenando os dados da wdi e do BNDES para aplcar a clusterização

In [ ]:
setor = "industria"

aux_wdi = wdi_t[4:].loc[:,pd.IndexSlice[:, relacao_01[setor][1]]]
aux_wdi = aux_wdi.droplevel(1, axis=1)
aux_wdi = aux_wdi.fillna(0)
aux_wdi.index = [i[0] for i in aux_wdi.index.to_flat_index()]
aux_wdi.index = aux_wdi.index.astype(int)
aux_wdi = aux_wdi.loc[lista_anos_interesse.astype(int)]

concat = pd.concat([aux_wdi, pd.DataFrame({'BNDES': df_desembolsos_organizado[setor]})], axis=1)
concat.head()

# Clusterizando

In [ ]:
#ANALISE DO NUMERO DE CLUSTER
def cidade_grupo(nclusters, cidade='BNDES'):
    pais_index = list(concat.columns).index(cidade) 
    
    clustering = KMeans(n_clusters=nclusters,algorithm='full').fit(concat.values.T)
    lista_paises = concat.columns[np.where(clustering.labels_ == clustering.labels_[pais_index])[0]]
    return set(lista_paises)

paises_similares_dict = {} 
ncls = []
n_clusters = np.array(range(10,30))
for k in n_clusters:
    listpaises = cidade_grupo(k)
    paises_similares_dict[str(k)] = list(listpaises)
    ncls.append(len(listpaises))
    
plt.bar(range(len(ncls)),ncls)
plt.xticks(range(len(n_clusters)), range(n_clusters.min(),n_clusters.max() + 1))
plt.title('Tot. de países no cluster do BNDES x num. de clusters')
plt.xlabel('Numero de clusters')
plt.ylabel('Total de paises no cluster do BNDES')
plt.show()

In [ ]:
countries = np.array(paises_similares_dict['25'])

fig = go.Figure([go.Scatter(
    x=concat.index, 
    y=concat[i], name=i) 
    for i in countries])
fig.show()